### Import modules

In [1]:
import numpy as np
import pandas as pd
import pickle
import lxml

import bs4 as bs
import urllib.request
import json
from bs4 import BeautifulSoup
import requests
import re
from collections import Counter
import http.client as http

import MySQLdb, pickle
from sqlalchemy import create_engine

### Crawling data from Cars.com

In [15]:
# Create a data frame to store crawling data
df = pd.DataFrame(columns = ['title','year','brand','model','miles','star','review_no','price'])

# Getting data from cars.com
for page in range(0,50):
    # Cars.com URL
    url = 'https://www.cars.com/for-sale/searchresults.action/?page='+str(page)+'&perPage=100&searchSource=PAGINATION&shippable-dealers-checkbox=true&sort=relevance&stkTypId=28881&zc=30309&localVehicles=false'  
    
    try:
        # Requesting data 
        sauce = urllib.request.urlopen(url).read()
        
    except (http.IncompleteRead) as e:
        sauce = e.partial
    
    # Parsing data with lxml
    soup = bs.BeautifulSoup(sauce, 'html.parser')
#     print(soup)
    
    # All of the data comes with listing-row__details as the class name
    specificSoup = soup.find_all('div', class_='vehicle-card')
#     print(specificSoup)

    
    # Put the imported data in the data frame
    for div in specificSoup:
#         print(div)
        
        # Put information about the name of the car in the name variable
        try:
            title = div.find('h2', {'class' :'title'}).text
#             print("title",title)
        
#             name = name_index.split("\n")
#             print("name",name)
        
            # Get only the car's year out of the car information in the name variable
            year_index = re.findall('[0-9]{4}',title)[0:1]
            year = year_index[0]
#             print("year",year)

#             # Get only the car's title out of the car information in the name variable
#             title_index = name.split(" ")[29:34]
#             title = " ".join(title_index)
#             print(title)

            # Get the car's brand out of the car information in the title variable
            brand = title.split(" ")[1]
#             print("brand",brand)
        
            # If there hasn't model information, model will be as a brand
            try:
                model = title.split(" ")[2:]
                model = " ".join(model)
#                 print("model",model)
            except:
                model = brand

#             # The miles of the car 
            mile_index = div.find('div', {'class' : 'mileage'}).text   
#             print("mile_index",mile_index)
            miles = mile_index.split(" ")[0].strip()
#             print("miles", miles)
            regex = re.compile("\d+")
            miles = regex.findall(miles) 
            miles = ''.join(miles)
            miles = miles.strip()
#             print("miles", miles)
        
            if miles == "":
                miles = 0
            else:
                miles

#             # Exterior color
#             exterior_color = div.find('ul', {'class' : 'listing-row__meta'}).text
#             try:
#                 exterior_color = re.sub('\n', ' ',exterior_color).split(" ")[45]
#             except:
#                 exterior_color = 'black'

    #         print(exterior_color)

#             # Interior color
#             interior_color = div.find('ul', {'class' : 'listing-row__meta'}).text
#             try:
#                 interior_color = re.sub('\n', ' ',interior_color).split(" ")[119]
#             except:
#                 interior_color = "black"
#     #         print(interior_color)

            # Transmission type
#             transmission = div.find('ul', {'class' : 'listing-row__meta'})
#             try:
#                 transmission = list(transmission)[5].text.split(" ")[40]
#                 transmission = re.sub('\n', ' ',transmission)
#             except:
#                 transmission = "6-speed"


#             # Drivetrain type
#             drivetrain = div.find('ul', {'class' : 'listing-row__meta'})
#             try:
#                 drivetrain = list(drivetrain)[7].text.split(" ")[40]
#                 drivetrain = re.sub('\n', ' ',drivetrain)
#             except: 
#                 drivetrain = 'fwd'
#             if drivetrain == 'rear wheel drive':
#                 drivetrain = 'rwd'
#             elif drivetrain == 'front wheel drive':
#                 drivetrain = 'fwd'
#             elif drivetrain == 'Unknown':
#                 drivetrain = 'fwd'
#             elif drivetrain == 'all wheel drive':
#                 drivetrain = '4wd'
#             elif drivetrain == '4x4/4-wheel':
#                 drivetrain = '4wd'           
            
#        Number of star
            if div.find('span',{'class' : 'sds-rating__count'}) == None:
                star = 0.0
            else:
                star_index =div.find('span',{'class' : 'sds-rating__count'}).text
                star = float(star_index)
#                 print(star)
#             star = star_index.split(" ")[40]
#             regex = re.compile("\d")
#             star = regex.findall(star)[0]

             # Number of review
            if div.find('span',{'class' : 'sds-rating__link sds-button-link'}) == None:
                review_no = 0
            else:
                review_index =div.find('span',{'class' : 'sds-rating__link sds-button-link'}).text
#                 print("review_index",review_index)
                review_no = re.sub('\n', '',review_index.split(" ")[0])
#                 print("review_no",review_no)
                try:
                    review_no = re.sub('[(,)]+', '',review_no)
                    review_no = int(review_no)
#                     print("review_no",review_no)
                except:
                    review_no = review_no
#                     print(review_no)

            # Car price
            if div.find('span', {'class' : 'primary-price'}) == None:
                price = 0
            else:
                price_index = div.find('span', {'class' : 'primary-price'}).text
                price = price_index.strip()
                regex = re.compile("\d")
                price = ''.join(regex.findall(price))
                price = int(price)
#                 print(price)


            # Entering crawled data into a data frame
            data = { 
                    'year' : year,
                    'title' : title.lower(),
                    'brand': brand.lower(),
                    'model': model.lower(),
                    'miles' : miles,
#                     'exterior_color' : exterior_color.lower(),
#                     'interior_color' : interior_color.lower(),
#                     'transmission' : transmission.lower(),
#                     'drivetrain' : drivetrain.lower(),
                    'star': star,
                    'review_no' : review_no,
                    'price': price,
                        }


            df.loc[len(df)] = data
        except:
            continue
            
df


,title,year,brand,model,miles,star,review_no,price
0,2018 ford f-150 platinum,2018,ford,f-150 platinum,58432,4.9,1438,51995
1,2018 ford f-150 platinum,2018,ford,f-150 platinum,45277,4.5,464,51125
2,2015 lexus gx 460 base,2015,lexus,gx 460 base,97028,2.8,28,31992
3,2019 bmw 740 i,2019,bmw,740 i,26535,4.5,642,56995
4,2018 bmw x2 sdrive28i,2018,bmw,x2 sdrive28i,14693,4.1,569,33500
...,...,...,...,...,...,...,...,...
995,2019 bmw x5 xdrive40i,2019,bmw,x5 xdrive40i,15010,4.5,642,60995
996,2014 chevrolet corvette stingray z51,2014,chevrolet,corvette stingray z51,48613,4.0,42,46859
997,2021 toyota 4runner limited,2021,toyota,4runner limited,18088,4.3,362,51995
998,2018 bmw x2 xdrive28i,2018,bmw,x2 xdrive28i,29259,4.5,642,32995


In [16]:
# Exclude prices and non-branded data
df = df[df["price"] != ""]
df = df[df["brand"] != ""]

In [17]:
# Transforming data for data processing
df["year"] = df["year"].astype('int')
df["miles"] = df["miles"].astype('int')
# df["star"] = df["star"].astype('int')
df["review_no"] = df["review_no"].astype('int')
df["price"] = df["price"].astype('int')

### Put Crawled data into Database (MySQL) at Amazon web service cloud

In [18]:
# Load pickle file as database password
pw = pickle.load(open('./Flask/models/pw.plk','rb'))

In [19]:
# Saving data to the database(MySQL at Amazon Web Service Cloud )
engine = create_engine("mysql+mysqldb://root:" + pw +"@ec2-34-219-13-119.us-west-2.compute.amazonaws.com/car", pool_pre_ping=True)

In [ ]:
 df.to_sql(name="car", con=engine, if_exists='replace')